In [1]:
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from metpy.units import units
from siphon.catalog import TDSCatalog
import numpy as np
import xarray as xr

In [4]:
# Get datetime from computer closest to now
date = datetime(2025, 2, 22, 0)
#date = datetime.utcnow()

# Create variables for URL generation
region = 'CONUS'
channel = 2
satellite = 'east'

In [5]:
def get_satellite_data(date, region='CONUS', channel=2, satellite='east'):
    '''
    This function helps to obtain GOES satellite data by channel from the UCAR THREDDS server.
    
    Input:
      * date - a datetime object specifying the desired time
      * region - the portion of satellite data wanted
        - CONUS (default)
        - fulldisk
      * channel (int value 1 - 16; default is 2)
      * satellite ('east' or 'west'; default is 'east')
    
    Output:
      * xarray DataArray containing the nearest time requested satellite data for the region, channel, and satellite.
    '''
    # We want to match something like:
    # https://thredds-test.unidata.ucar.edu/thredds/catalog/satellite/goes16/GOES16/Mesoscale-1/Channel08/20181113/catalog.html

    # Construct the data_url string
    data_url = (f'https://thredds.ucar.edu/thredds/catalog/satellite/goes/{satellite}/products/'
                f'CloudAndMoistureImagery/{region}/Channel{channel:02d}/'
                f'{date:%Y%m%d}/catalog.xml')

    # Get list of files available for particular day
    cat = TDSCatalog(data_url)

    # Grab dataset for desired time
    dataset = cat.datasets.filter_time_nearest(date, regex=r'_s(?P<strptime>\d{13})', strptime='%Y%j%H%M%S')

    # Open most recent file available
    ds = dataset.remote_access(use_xarray=True,service='OPENDAP').metpy.parse_cf()
    
    return ds

In [6]:
# Call function to get data closest to requested time
ds = get_satellite_data(date, region, channel, satellite)

#Apply a square root correction for visible imagery only
if channel == 2:
    ds['Sectorized_CMI'].values = np.sqrt(ds['Sectorized_CMI'].values)

# specify the valid time from the file
vtime = ds.time.values.astype('datetime64[ms]').astype('O')
print(vtime)

2025-02-22 00:01:17


In [7]:
ds

<xarray.Dataset> Size: 240MB
Dimensions:               (y: 6000, x: 10000)
Coordinates:
    time                  datetime64[ns] 8B 2025-02-22T00:01:17
  * y                     (y) float64 48kB 4.589e+06 4.588e+06 ... 1.583e+06
  * x                     (x) float64 80kB -3.627e+06 -3.627e+06 ... 1.383e+06
    metpy_crs             object 8B Projection: geostationary
Data variables:
    fixedgrid_projection  int32 4B ...
    Sectorized_CMI        (y, x) float32 240MB nan nan nan ... 0.03086 nan
Attributes: (12/30)
    title:                       Sectorized Cloud and Moisture Imagery for th...
    ICD_version:                 GROUND SEGMENT (GS) TO ADVANCED WEATHER INTE...
    Conventions:                 CF-1.6
    channel_id:                  2
    central_wavelength:          0.64
    abi_mode:                    6
    ...                          ...
    pixel_y_size:                0.5
    satellite_latitude:          0.0
    satellite_longitude:         -75.0
    satellite_altitude:          35786023.0
    created_by:                  ldm-alchemy
    product_tiles_received:      15

In [8]:
# Pull projection from DataArray
proj = ds.Sectorized_CMI.metpy.cartopy_crs

# We also need the x (north/south) and y (east/west) axis sweep of the ABI data
x = ds.Sectorized_CMI.x
y = ds.Sectorized_CMI.y
#proj
#x
#y